In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#출력 그림 크기 조절
sns.set(rc={'figure.figsize':(12,12)})
plt.style.use('ggplot')
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

import warnings
warnings.filterwarnings("ignore")

# 3. 데이터 전처리

- 진행사항

1. 버스 이상치 0으로 임퓨테이션?
 ->1) 결측치 경상남도에 유일하게 존재-> 임대료 평균으로 대치->경상남도에 비슷한 값 없어서 경상북도 값으로 대치(train에서 경상남도,경상북도 분포 비슷함)
 ->2) train 아웃라이어 99퀀타일값(15)으로 대치(분포를 유지하는 선에서)
      test  아웃라이어 위와 같은 이유로 19로 대치

2. 지하철 결측치 전처리
 ->1) “경상남도”, “충청남도” 지역에는 지하철이 존재하지 않음 -> 0으로 대치
 ->2) “대전광역시” 지역 결측치는 일단 0으로 대치.(**추후에 1로 변경 가능**)
3. 열이름 변환 -  지하철(11),버스정류장(12) 열이름 변환
4. test 데이터 자격유형 결측치 대치 ( 그룹화 / 국민임대 - 자격유형별 - 임대료)

In [2]:
# 데이터 로드
age_gender = pd.read_csv('data/age_gender_info.csv')


train_m = pd.read_csv('data/trainm0710(1).csv', encoding = 'ansi')
test_m = pd.read_csv('data/testm0710(1).csv', encoding = 'ansi')

In [3]:
train_m

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,...,60F,60M,70F,70M,80F,80M,90F,90M,100F,100M
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17,A,9216000,...,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17,A,12672000,...,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17,A,12672000,...,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17,A,18433000,...,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17,A,18433000,...,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2864,C2532,239,아파트,강원도,국민임대,49.20,19,7,A,11346000,...,0.088423,0.070014,0.047732,0.032761,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
2865,C2532,239,아파트,강원도,국민임대,51.08,34,7,A,14005000,...,0.088423,0.070014,0.047732,0.032761,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
2866,C2532,239,아파트,강원도,국민임대,51.73,34,7,A,14005000,...,0.088423,0.070014,0.047732,0.032761,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
2867,C2532,239,아파트,강원도,국민임대,51.96,114,7,A,14005000,...,0.088423,0.070014,0.047732,0.032761,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017


## 3. 1 결측치 처리

### 3.1.1 임대보증금, 임대료

- 결측치 train(nan = 569, '-' = ), test(177)
- 상가는 임대보증금, 임대료가 전부 결측 => 상가의 임대보증금, 임대료는 모두 0으로 대치해도 될듯 => ***학습시 이미 상가라는 부분에서 한 번 걸러질듯***
- 임대보증금, 임대료 부분,,, 모두 공공분양, 임대상가, 자격유형은 모두 D 조건 및 특징 확인 후 전처리 결측치 처리,,,
- ***일단 모두 0으로 대치하고 진행?? 좀 더 좋은 생각이 있으면 바꾸기***

#### train
- 국민임대, 행복주택의 경우 결측치가 없는 행이 존재 => 좀 더 생각
- 장기전세는 1가지 경우에만 존재 따라서 결측치 0 으로 대치 가능
- 공공분양은 1가지 경우에만 존재 따라서 결측치 0 으로 대치 가능

#### test
- 영구임대, 행복주택의 경우 결측치가 없는 행이 존재 => 좀 더 생각

In [4]:
train_m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2869 entries, 0 to 2868
Data columns (total 37 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   단지코드      2869 non-null   object 
 1   총세대수      2869 non-null   int64  
 2   임대건물구분    2869 non-null   object 
 3   지역        2869 non-null   object 
 4   공급유형      2869 non-null   object 
 5   전용면적      2869 non-null   float64
 6   전용면적별세대수  2869 non-null   int64  
 7   공가수       2869 non-null   int64  
 8   자격유형      2869 non-null   object 
 9   임대보증금     2869 non-null   int64  
 10  임대료       2869 non-null   int64  
 11  지하철역수     2869 non-null   int64  
 12  버스정류장수    2869 non-null   int64  
 13  단지내주차면수   2869 non-null   int64  
 14  등록차량수     2869 non-null   int64  
 15  L10F      2869 non-null   float64
 16  L10M      2869 non-null   float64
 17  10F       2869 non-null   float64
 18  10M       2869 non-null   float64
 19  20F       2869 non-null   float64
 20  20M       2869 non-null   floa

In [5]:
test_m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 36 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   단지코드      1008 non-null   object 
 1   총세대수      1008 non-null   int64  
 2   임대건물구분    1008 non-null   object 
 3   지역        1008 non-null   object 
 4   공급유형      1008 non-null   object 
 5   전용면적      1008 non-null   float64
 6   전용면적별세대수  1008 non-null   int64  
 7   공가수       1008 non-null   int64  
 8   자격유형      1008 non-null   object 
 9   임대보증금     1008 non-null   int64  
 10  임대료       1008 non-null   int64  
 11  지하철역수     1008 non-null   int64  
 12  버스정류장수    1008 non-null   int64  
 13  단지내주차면수   1008 non-null   int64  
 14  L10F      1008 non-null   float64
 15  L10M      1008 non-null   float64
 16  10F       1008 non-null   float64
 17  10M       1008 non-null   float64
 18  20F       1008 non-null   float64
 19  20M       1008 non-null   float64
 20  30F       1008 non-null   floa

In [6]:
train_o = train_m.loc[train_m["임대료"] == 0,:"단지내주차면수"]
test_o = test_m.loc[test_m["임대료"] == 0,:"단지내주차면수"]

In [7]:
train_o[train_o["공급유형"] != "임대상가"]
#장기전세, 공공분양, 국민임대, 행복주택

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철역수,버스정류장수,단지내주차면수
2073,C1039,790,아파트,서울특별시,장기전세,51.32,126,13,A,187694000,0,0,3,673
2074,C1039,790,아파트,서울특별시,장기전세,59.88,49,13,A,213863000,0,0,3,673
2075,C1039,790,아파트,서울특별시,장기전세,59.94,75,13,A,213863000,0,0,3,673
2284,C1350,1401,아파트,대전광역시,공공분양,74.94,317,2,D,0,0,0,6,1636
2285,C1350,1401,아파트,대전광역시,공공분양,74.94,137,2,D,0,0,0,6,1636
2286,C1350,1401,아파트,대전광역시,공공분양,74.94,22,2,D,0,0,0,6,1636
2287,C1350,1401,아파트,대전광역시,공공분양,84.94,164,2,D,0,0,0,6,1636
2288,C1350,1401,아파트,대전광역시,공공분양,84.94,19,2,D,0,0,0,6,1636
2289,C1350,1401,아파트,대전광역시,공공분양,84.96,26,2,D,0,0,0,6,1636
2290,C1350,1401,아파트,대전광역시,공공분양,84.97,26,2,D,0,0,0,6,1636


In [8]:
test_o[test_o["공급유형"] != "임대상가"]
#영구임대, 행복주택

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철역수,버스정류장수,단지내주차면수
76,C1006,1505,아파트,대전광역시,영구임대,26.37,10,27,D,0,0,2,5,428
77,C1006,1505,아파트,대전광역시,영구임대,26.37,10,27,D,0,0,2,5,428
79,C1006,1505,아파트,대전광역시,영구임대,52.74,6,27,D,0,0,2,5,428
991,C2152,120,아파트,강원도,영구임대,24.83,66,9,C,0,0,0,1,40
992,C2152,120,아파트,강원도,영구임대,33.84,54,9,C,0,0,0,1,40
1000,C1267,675,아파트,경상남도,행복주택,16.94,50,38,L,0,0,0,1,467
1001,C1267,675,아파트,경상남도,행복주택,26.85,66,38,L,0,0,0,1,467
1002,C1267,675,아파트,경상남도,행복주택,26.85,8,38,L,0,0,0,1,467
1003,C1267,675,아파트,경상남도,행복주택,36.77,126,38,L,0,0,0,1,467


In [9]:
train_m.loc[train_m["공급유형"] == "공공분양",:"단지내주차면수"]
#공공분양은 1가지 경우에만 존재 따라서 결측치 0 으로 대치 가능

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철역수,버스정류장수,단지내주차면수
2284,C1350,1401,아파트,대전광역시,공공분양,74.94,317,2,D,0,0,0,6,1636
2285,C1350,1401,아파트,대전광역시,공공분양,74.94,137,2,D,0,0,0,6,1636
2286,C1350,1401,아파트,대전광역시,공공분양,74.94,22,2,D,0,0,0,6,1636
2287,C1350,1401,아파트,대전광역시,공공분양,84.94,164,2,D,0,0,0,6,1636
2288,C1350,1401,아파트,대전광역시,공공분양,84.94,19,2,D,0,0,0,6,1636
2289,C1350,1401,아파트,대전광역시,공공분양,84.96,26,2,D,0,0,0,6,1636
2290,C1350,1401,아파트,대전광역시,공공분양,84.97,26,2,D,0,0,0,6,1636


In [10]:
train_m.loc[train_m["공급유형"] == "장기전세",:"단지내주차면수"]
#장기전세는 1가지 경우에만 존재 따라서 결측치 0 으로 대치 가능

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철역수,버스정류장수,단지내주차면수
2073,C1039,790,아파트,서울특별시,장기전세,51.32,126,13,A,187694000,0,0,3,673
2074,C1039,790,아파트,서울특별시,장기전세,59.88,49,13,A,213863000,0,0,3,673
2075,C1039,790,아파트,서울특별시,장기전세,59.94,75,13,A,213863000,0,0,3,673


In [11]:
train_m.loc[train_m["공급유형"] == "국민임대",:"단지내주차면수"]
#국민임대, 행복주택의 경우 결측치가 없는 행이 존재 => 좀 더 생각

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철역수,버스정류장수,단지내주차면수
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17,A,9216000,82940,0,3,624
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17,A,12672000,107130,0,3,624
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17,A,12672000,107130,0,3,624
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17,A,18433000,149760,0,3,624
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17,A,18433000,149760,0,3,624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2864,C2532,239,아파트,강원도,국민임대,49.20,19,7,A,11346000,116090,0,1,166
2865,C2532,239,아파트,강원도,국민임대,51.08,34,7,A,14005000,142310,0,1,166
2866,C2532,239,아파트,강원도,국민임대,51.73,34,7,A,14005000,142310,0,1,166
2867,C2532,239,아파트,강원도,국민임대,51.96,114,7,A,14005000,142310,0,1,166


In [12]:
test_m.loc[test_m["공급유형"] == "영구임대",:"단지내주차면수"]
#국민임대, 행복주택의 경우 결측치가 없는 행이 존재 => 좀 더 생각

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철역수,버스정류장수,단지내주차면수
73,C1006,1505,아파트,대전광역시,영구임대,26.37,358,27,C,5787000,79980,2,5,428
74,C1006,1505,아파트,대전광역시,영구임대,26.37,229,27,C,5787000,79980,2,5,428
75,C1006,1505,아파트,대전광역시,영구임대,26.37,574,27,C,5787000,79980,2,5,428
76,C1006,1505,아파트,대전광역시,영구임대,26.37,10,27,D,0,0,2,5,428
77,C1006,1505,아파트,대전광역시,영구임대,26.37,10,27,D,0,0,2,5,428
78,C1006,1505,아파트,대전광역시,영구임대,31.32,298,27,C,6873000,94990,2,5,428
79,C1006,1505,아파트,대전광역시,영구임대,52.74,6,27,D,0,0,2,5,428
113,C2676,2572,아파트,부산광역시,영구임대,26.37,294,25,C,5787000,79980,0,2,418
114,C2676,2572,아파트,부산광역시,영구임대,26.37,1341,25,C,5787000,79980,0,2,418
115,C2676,2572,아파트,부산광역시,영구임대,31.32,894,25,C,6873000,94990,0,2,418


In [13]:
test_m.loc[test_m["공급유형"] == "행복주택",:"단지내주차면수"]
#영구임대, 행복주택의 경우 결측치가 없는 행이 존재 => 좀 더 생각

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철역수,버스정류장수,단지내주차면수
846,C1003,480,아파트,경상남도,행복주택,16.69,128,29,J,12000000,61000,0,3,339
847,C1003,480,아파트,경상남도,행복주택,26.34,172,29,J,19600000,100000,0,3,339
848,C1003,480,아파트,경상남도,행복주택,26.34,48,29,J,19600000,100000,0,3,339
849,C1003,480,아파트,경상남도,행복주택,26.52,30,29,J,18800000,96000,0,3,339
850,C1003,480,아파트,경상남도,행복주택,36.31,56,29,J,25600000,131000,0,3,339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,C1038,554,아파트,충청북도,행복주택,36.95,18,34,K,79833000,41580,0,1,387
1000,C1267,675,아파트,경상남도,행복주택,16.94,50,38,L,0,0,0,1,467
1001,C1267,675,아파트,경상남도,행복주택,26.85,66,38,L,0,0,0,1,467
1002,C1267,675,아파트,경상남도,행복주택,26.85,8,38,L,0,0,0,1,467


### 3.1.2 Train, Test 차집합
- Train 셋에는 존재하지만 Test셋에는 존재 하지 않는 데이터 값 존재
- Train 데이터 기준 차집합 지역 : '서울특별시'
- Train 데이터 기준 차집합 공급유형 : '공공분양', '공공임대(5년)', '장기전세' => "이외"
- Train 데이터 기준 차집합 자격유형 : 'O', 'F', 'B' => "이외"

- **우리의 목표는 test데이터에 대한 오류치 최소화 따라서 제거하여 노이즈가 될 여지를 방지** 
- ***확인 결과 100개 좀 안되는 데이터....*** 

In [14]:
# train 서울특별시
train_m.loc[train_m["지역"] == "서울특별시"] #50개 데이터

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,...,60F,60M,70F,70M,80F,80M,90F,90M,100F,100M
1257,C2340,512,아파트,서울특별시,공공임대(50년),39.82,512,15,A,32454000,...,0.119639,0.078939,0.097543,0.06105,0.047853,0.023463,0.011344,0.003326,0.000532,0.000152
1936,C1344,424,아파트,서울특별시,공공임대(10년),51.89,35,0,A,50758000,...,0.119639,0.078939,0.097543,0.06105,0.047853,0.023463,0.011344,0.003326,0.000532,0.000152
1937,C1344,424,아파트,서울특별시,공공임대(10년),59.93,43,0,A,63166000,...,0.119639,0.078939,0.097543,0.06105,0.047853,0.023463,0.011344,0.003326,0.000532,0.000152
1938,C1344,424,아파트,서울특별시,공공임대(분납),74.97,41,0,A,125326000,...,0.119639,0.078939,0.097543,0.06105,0.047853,0.023463,0.011344,0.003326,0.000532,0.000152
1939,C1344,424,아파트,서울특별시,공공임대(분납),84.95,39,0,A,143545000,...,0.119639,0.078939,0.097543,0.06105,0.047853,0.023463,0.011344,0.003326,0.000532,0.000152
1940,C1344,424,아파트,서울특별시,공공임대(분납),84.97,42,0,A,145264000,...,0.119639,0.078939,0.097543,0.06105,0.047853,0.023463,0.011344,0.003326,0.000532,0.000152
1941,C1344,424,아파트,서울특별시,공공임대(분납),84.99,100,0,A,142744000,...,0.119639,0.078939,0.097543,0.06105,0.047853,0.023463,0.011344,0.003326,0.000532,0.000152
2050,C2352,1065,아파트,서울특별시,국민임대,36.05,25,7,A,21015000,...,0.119639,0.078939,0.097543,0.06105,0.047853,0.023463,0.011344,0.003326,0.000532,0.000152
2051,C2352,1065,아파트,서울특별시,국민임대,36.08,448,7,A,21015000,...,0.119639,0.078939,0.097543,0.06105,0.047853,0.023463,0.011344,0.003326,0.000532,0.000152
2052,C2352,1065,아파트,서울특별시,국민임대,36.62,84,7,A,21015000,...,0.119639,0.078939,0.097543,0.06105,0.047853,0.023463,0.011344,0.003326,0.000532,0.000152


In [15]:
train_m.loc[train_m["공급유형"] == "공공분양"] #7개 데이터

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,...,60F,60M,70F,70M,80F,80M,90F,90M,100F,100M
2284,C1350,1401,아파트,대전광역시,공공분양,74.94,317,2,D,0,...,0.088468,0.070261,0.05101,0.037143,0.032455,0.013751,0.006494,0.00174,0.000298,0.000066
2285,C1350,1401,아파트,대전광역시,공공분양,74.94,137,2,D,0,...,0.088468,0.070261,0.05101,0.037143,0.032455,0.013751,0.006494,0.00174,0.000298,0.000066
2286,C1350,1401,아파트,대전광역시,공공분양,74.94,22,2,D,0,...,0.088468,0.070261,0.05101,0.037143,0.032455,0.013751,0.006494,0.00174,0.000298,0.000066
2287,C1350,1401,아파트,대전광역시,공공분양,84.94,164,2,D,0,...,0.088468,0.070261,0.05101,0.037143,0.032455,0.013751,0.006494,0.00174,0.000298,0.000066
2288,C1350,1401,아파트,대전광역시,공공분양,84.94,19,2,D,0,...,0.088468,0.070261,0.05101,0.037143,0.032455,0.013751,0.006494,0.00174,0.000298,0.000066
2289,C1350,1401,아파트,대전광역시,공공분양,84.96,26,2,D,0,...,0.088468,0.070261,0.05101,0.037143,0.032455,0.013751,0.006494,0.00174,0.000298,0.000066
2290,C1350,1401,아파트,대전광역시,공공분양,84.97,26,2,D,0,...,0.088468,0.070261,0.05101,0.037143,0.032455,0.013751,0.006494,0.00174,0.000298,0.000066


In [16]:
train_m.loc[train_m["공급유형"] == "공공임대(5년)"] # 3개 데이터

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,...,60F,60M,70F,70M,80F,80M,90F,90M,100F,100M
2508,C2470,636,아파트,대전광역시,공공임대(5년),39.90,96,1,A,23000000,...,0.088468,0.070261,0.05101,0.037143,0.032455,0.013751,0.006494,0.00174,0.000298,0.000066
2509,C2470,636,아파트,대전광역시,공공임대(5년),46.81,264,1,A,26000000,...,0.088468,0.070261,0.05101,0.037143,0.032455,0.013751,0.006494,0.00174,0.000298,0.000066
2510,C2470,636,아파트,대전광역시,공공임대(5년),59.95,46,1,A,32000000,...,0.088468,0.070261,0.05101,0.037143,0.032455,0.013751,0.006494,0.00174,0.000298,0.000066


In [17]:
train_m.loc[train_m["공급유형"] == "장기전세"] #3개

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,...,60F,60M,70F,70M,80F,80M,90F,90M,100F,100M
2073,C1039,790,아파트,서울특별시,장기전세,51.32,126,13,A,187694000,...,0.119639,0.078939,0.097543,0.06105,0.047853,0.023463,0.011344,0.003326,0.000532,0.000152
2074,C1039,790,아파트,서울특별시,장기전세,59.88,49,13,A,213863000,...,0.119639,0.078939,0.097543,0.06105,0.047853,0.023463,0.011344,0.003326,0.000532,0.000152
2075,C1039,790,아파트,서울특별시,장기전세,59.94,75,13,A,213863000,...,0.119639,0.078939,0.097543,0.06105,0.047853,0.023463,0.011344,0.003326,0.000532,0.000152


In [18]:
for i in ["O","B","F"]:
    display(train_m.loc[train_m["자격유형"] == i]) # O = 1,B = 21,F = 3

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,...,60F,60M,70F,70M,80F,80M,90F,90M,100F,100M
2535,C1659,902,아파트,광주광역시,행복주택,44.78,48,32,O,41120000,...,0.07593,0.059586,0.048552,0.031754,0.029749,0.010341,0.006343,0.000895,0.000353,0.000013


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,...,60F,60M,70F,70M,80F,80M,90F,90M,100F,100M
18,C1945,755,아파트,경기도,국민임대,39.72,120,6,B,15607000,...,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
19,C1945,755,아파트,경기도,국민임대,39.72,120,6,B,15607000,...,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
20,C1945,755,아파트,경기도,국민임대,51.93,207,6,B,23139000,...,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
21,C1945,755,아파트,경기도,국민임대,51.93,96,6,B,23139000,...,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
22,C1945,755,아파트,경기도,국민임대,59.88,160,6,B,29209000,...,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
23,C1945,755,아파트,경기도,국민임대,59.88,52,6,B,29209000,...,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
1177,C2066,757,아파트,대구광역시,국민임대,36.62,217,22,B,13007000,...,0.094511,0.073133,0.055914,0.036572,0.031887,0.012556,0.005364,0.001259,0.000219,0.000000
1178,C2066,757,아파트,대구광역시,국민임대,36.62,69,22,B,13007000,...,0.094511,0.073133,0.055914,0.036572,0.031887,0.012556,0.005364,0.001259,0.000219,0.000000
1179,C2066,757,아파트,대구광역시,국민임대,36.62,51,22,B,13007000,...,0.094511,0.073133,0.055914,0.036572,0.031887,0.012556,0.005364,0.001259,0.000219,0.000000
1180,C2066,757,아파트,대구광역시,국민임대,39.68,140,22,B,14863000,...,0.094511,0.073133,0.055914,0.036572,0.031887,0.012556,0.005364,0.001259,0.000219,0.000000


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,...,60F,60M,70F,70M,80F,80M,90F,90M,100F,100M
514,C2132,2428,아파트,부산광역시,영구임대,26.37,358,1,F,3912000,...,0.109297,0.085294,0.078743,0.053388,0.047908,0.020228,0.008043,0.00224,0.000268,0.000028
515,C2132,2428,아파트,부산광역시,영구임대,26.37,1192,1,F,3912000,...,0.109297,0.085294,0.078743,0.053388,0.047908,0.020228,0.008043,0.00224,0.000268,0.000028
517,C2132,2428,아파트,부산광역시,영구임대,31.32,596,1,F,4646000,...,0.109297,0.085294,0.078743,0.053388,0.047908,0.020228,0.008043,0.00224,0.000268,0.000028


#### 차집합 제거

In [19]:
#제거하기
#train_m = train_m.loc[train_m.지역 != '서울특별시']

#mask = train_m.공급유형 != '공공분양'
#mask &= train_m.공급유형 != '장기전세'
#mask &= train_m.공급유형 != '공공임대(5년)'
#train_m = train_m.loc[mask]

#mask = train_m.자격유형 != 'F'
#mask &= train_m.자격유형 != 'B'
#mask &= train_m.자격유형 != 'O'
#train_m = train_m.loc[mask]

## 3.2 변수 재정의

- 변수 재정의에 앞서 새 데이터프레임 생성


In [20]:
train_m.groupby(['단지코드']).nunique(dropna=False)

,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,...,60F,60M,70F,70M,80F,80M,90F,90M,100F,100M
단지코드,,,,,,,,,,,,,,,,,,,,,
C1000,1,1,1,1,5,6,1,1,3,3,...,1,1,1,1,1,1,1,1,1,1
C1004,1,2,1,2,15,3,1,2,3,3,...,1,1,1,1,1,1,1,1,1,1
C1005,1,1,1,1,3,3,1,1,3,3,...,1,1,1,1,1,1,1,1,1,1
C1013,1,1,1,1,4,5,1,1,3,3,...,1,1,1,1,1,1,1,1,1,1
C1014,1,1,1,1,6,7,1,1,4,4,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C2663,1,1,1,1,5,6,1,1,4,4,...,1,1,1,1,1,1,1,1,1,1
C2666,1,1,1,1,4,6,1,1,2,2,...,1,1,1,1,1,1,1,1,1,1
C2670,1,1,1,1,4,3,1,1,3,3,...,1,1,1,1,1,1,1,1,1,1


In [21]:
train_m.groupby(['단지코드']).nunique(dropna=False).sum(axis=0)

총세대수         411
임대건물구분       444
지역           411
공급유형         475
전용면적        1835
전용면적별세대수    2155
공가수          411
자격유형         498
임대보증금       1234
임대료         1248
지하철역수        411
버스정류장수       411
단지내주차면수      411
등록차량수        411
L10F         411
L10M         411
10F          411
10M          411
20F          411
20M          411
30F          411
30M          411
40F          411
40M          411
50F          411
50M          411
60F          411
60M          411
70F          411
70M          411
80F          411
80M          411
90F          411
90M          411
100F         411
100M         411
dtype: int64

### 단지코드별 데이터 프레임 생성
- 유일값이 나오는 열은 따로 빼서 정리

In [22]:
unique_cols = ['총세대수', '지역', '공가수', 
               '지하철역수',
               '버스정류장수',
               '단지내주차면수', '등록차량수']
train = train_m.set_index('단지코드')[unique_cols].drop_duplicates()
test = test_m.set_index('단지코드')[[col for col in unique_cols if col!='등록차량수']].drop_duplicates()

In [23]:
train = train.reset_index()
test = test.reset_index()

In [24]:
train

,단지코드,총세대수,지역,공가수,지하철역수,버스정류장수,단지내주차면수,등록차량수
0,C2515,545,경상남도,17,0,3,624,205
1,C1407,1216,대전광역시,13,1,1,1285,1064
2,C1945,755,경기도,6,1,3,734,730
3,C1470,696,전라북도,14,0,2,645,553
4,C1898,566,전라북도,9,0,6,517,415
...,...,...,...,...,...,...,...,...
406,C2586,90,제주특별자치도,7,0,3,66,57
407,C2035,492,강원도,24,0,1,521,246
408,C2020,40,부산광역시,7,1,2,25,19
409,C2437,90,충청북도,12,0,1,30,16


### 3.2.1 임대건물구분
- 상가를 포함한 단지와 포함하지 않은 단지 구분? ***상가가 있는곳은 주차공간이 넓어야하지않나? 싶어서***
- 상가만 존재하는 단지가 있나? => x
- 따라서 상가 포함 단지와 상가 아파트 단독 단지를 구분
- 아파트 단독(1), 상가포함(2)

In [25]:
# 상가 보유 단지
print('전체 단지 수: ', train_m['단지코드'].nunique())
print('상가 보유 단지 수: ', train_m.loc[train_m['임대건물구분'] == '상가']['단지코드'].nunique())
print('상가 보유 단지: ', train_m.loc[train_m['임대건물구분'] == '상가']['단지코드'].unique())
print('상가 보유 단지 지역: ', train_m.loc[train_m['임대건물구분'] == '상가']['지역'].unique())

전체 단지 수:  411
상가 보유 단지 수:  33
상가 보유 단지:  ['C1925' 'C1874' 'C2416' 'C2621' 'C1616' 'C1704' 'C2258' 'C2038' 'C1859'
 'C1722' 'C2190' 'C1476' 'C1983' 'C2135' 'C2034' 'C1109' 'C2289' 'C2597'
 'C2310' 'C2132' 'C1439' 'C1899' 'C1056' 'C2644' 'C1206' 'C1775' 'C1790'
 'C2109' 'C1698' 'C1004' 'C1875' 'C2212' 'C2571']
상가 보유 단지 지역:  ['강원도' '충청남도' '경상남도' '대전광역시' '부산광역시' '제주특별자치도']


In [26]:
# 상가 보유 단지의 리스트로 상가만 존재하는 단지가 있는지 확인
temp_tr = ['C1925', 'C1874', 'C2416', 'C2621', 'C1616', 'C1704', 'C2258', 'C2038', 'C1859',
 'C1722', 'C2190', 'C1476', 'C1983', 'C2135', 'C2034', 'C1109', 'C2289', 'C2597',
 'C2310', 'C2132', 'C1439', 'C1899', 'C1056', 'C2644', 'C1206', 'C1775', 'C1790',
 'C2109', 'C1698', 'C1004', 'C1875', 'C2212', 'C2571']

for i in temp_tr:
    print(i)
    display(train_m.loc[train_m["단지코드"] == i]["임대건물구분"].unique())

C1925


array(['아파트', '상가'], dtype=object)

C1874


array(['아파트', '상가'], dtype=object)

C2416


...

array(['아파트', '상가'], dtype=object)

C2212


array(['아파트', '상가'], dtype=object)

C2571


array(['아파트', '상가'], dtype=object)

In [27]:
train["임대건물구분"] = "아파트"

#상가 보유하고 있으면 1, 아파트만 있으면 0
for i in temp_tr:
    train.loc[train["단지코드"] == i]["임대건물구분"] = "주상복합"

In [28]:
train

,단지코드,총세대수,지역,공가수,지하철역수,버스정류장수,단지내주차면수,등록차량수,임대건물구분
0,C2515,545,경상남도,17,0,3,624,205,아파트
1,C1407,1216,대전광역시,13,1,1,1285,1064,아파트
2,C1945,755,경기도,6,1,3,734,730,아파트
3,C1470,696,전라북도,14,0,2,645,553,아파트
4,C1898,566,전라북도,9,0,6,517,415,아파트
...,...,...,...,...,...,...,...,...,...
406,C2586,90,제주특별자치도,7,0,3,66,57,아파트
407,C2035,492,강원도,24,0,1,521,246,아파트
408,C2020,40,부산광역시,7,1,2,25,19,아파트
409,C2437,90,충청북도,12,0,1,30,16,아파트


In [29]:
# 상가 보유 단지
print('전체 단지 수: ', test_m['단지코드'].nunique())
print('상가 보유 단지 수: ', test_m.loc[test_m['임대건물구분'] == '상가']['단지코드'].nunique())
print('상가 보유 단지: ', test_m.loc[test_m['임대건물구분'] == '상가']['단지코드'].unique())
print('상가 보유 단지 지역: ', test_m.loc[test_m['임대건물구분'] == '상가']['지역'].unique())

전체 단지 수:  147
상가 보유 단지 수:  7
상가 보유 단지:  ['C1006' 'C2676' 'C2177' 'C2033' 'C1812' 'C2253' 'C1729']
상가 보유 단지 지역:  ['대전광역시' '부산광역시' '충청남도' '울산광역시' '강원도']


In [30]:
# 상가 보유 단지의 리스트로 상가만 존재하는 단지가 있는지 확인
temp_te = ['C1006', 'C2676', 'C2177', 'C2033', 'C1812', 'C2253', 'C1729']

for i in temp_te:
    print(i)
    display(test_m.loc[test_m["단지코드"] == i]["임대건물구분"].unique())

C1006


array(['아파트', '상가'], dtype=object)

C2676


array(['아파트', '상가'], dtype=object)

C2177


...

array(['아파트', '상가'], dtype=object)

C2253


array(['아파트', '상가'], dtype=object)

C1729


array(['아파트', '상가'], dtype=object)

In [31]:
test["임대건물구분"] = "아파트"

#상가 보유하고 있으면 1, 아파트만 있으면 0
for i in temp_te:
    test.loc[test["단지코드"] == i]["임대건물구분"] = "주상복합"

In [32]:
test

,단지코드,총세대수,지역,공가수,지하철역수,버스정류장수,단지내주차면수,임대건물구분
0,C1072,754,경기도,14,0,2,683,아파트
1,C1128,1354,경기도,9,0,3,1216,아파트
2,C1456,619,부산광역시,18,0,16,547,아파트
3,C1840,593,전라북도,7,0,3,543,아파트
4,C1332,1297,경기도,11,0,2,1112,아파트
...,...,...,...,...,...,...,...,...
142,C2456,349,제주특별자치도,17,0,4,270,아파트
143,C1266,596,충청북도,35,0,1,593,아파트
144,C2152,120,강원도,9,0,1,40,아파트
145,C1267,675,경상남도,38,0,1,467,아파트


In [33]:
train.loc[:,"임대건물구분"] = train.loc[:,"임대건물구분"].astype('category')
test.loc[:,"임대건물구분"] = test.loc[:,"임대건물구분"].astype('category')

### 3.2.2 주거율
- 주거율 : (1 - (공가수 / 총세대수))
- 주거율이 높으면 단지내 주차면수 대비 등록차량의 수의 비율이 높을것이라 예상

In [34]:
# 그 전에 미리 상가의 공가수?가 어떻게 되는지 확인
train_m[train_m["임대건물구분"] == "상가"]["공가수"]

80     9
81     9
82     9
83     9
93     2
      ..
826    5
827    5
828    5
829    5
830    5
Name: 공가수, Length: 562, dtype: int64

In [35]:
train["공가수"] = 1 - (train["공가수"]/train["총세대수"])
test["공가수"] = 1 - (test["공가수"]/test["총세대수"])

In [36]:
train.rename(columns={"공가수" : "주거율"}, inplace=True)
test.rename(columns={"공가수" : "주거율"}, inplace=True)

In [37]:
display(train.head(10),test.head(10))

,단지코드,총세대수,지역,주거율,지하철역수,버스정류장수,단지내주차면수,등록차량수,임대건물구분
0,C2515,545,경상남도,0.968807,0,3,624,205,아파트
1,C1407,1216,대전광역시,0.989309,1,1,1285,1064,아파트
2,C1945,755,경기도,0.992053,1,3,734,730,아파트
3,C1470,696,전라북도,0.979885,0,2,645,553,아파트
4,C1898,566,전라북도,0.984099,0,6,517,415,아파트
5,C1244,1722,경기도,0.994193,0,10,1483,1804,아파트
6,C1171,624,대전광역시,0.979167,0,5,634,700,아파트
7,C2073,361,강원도,0.963989,0,5,288,301,아파트
8,C2513,754,광주광역시,0.980106,0,6,530,820,아파트
9,C1936,240,광주광역시,0.966667,0,10,168,234,아파트


,단지코드,총세대수,지역,주거율,지하철역수,버스정류장수,단지내주차면수,임대건물구분
0,C1072,754,경기도,0.981432,0,2,683,아파트
1,C1128,1354,경기도,0.993353,0,3,1216,아파트
2,C1456,619,부산광역시,0.970921,0,16,547,아파트
3,C1840,593,전라북도,0.988196,0,3,543,아파트
4,C1332,1297,경기도,0.991519,0,2,1112,아파트
5,C1563,1974,경기도,0.992401,0,6,1696,아파트
6,C1794,1349,전라북도,0.981468,0,2,1098,아파트
7,C1640,533,전라북도,0.968105,0,1,470,아파트
8,C1377,470,경상남도,0.961702,0,4,384,아파트
9,C2072,353,경상남도,0.983003,0,1,280,아파트


### 3.2.1 면적
- 코드별로 묶은 후
- 전용 면적 비율

#### train

In [38]:
train_m["전용면적"]

0       33.48
1       39.60
2       39.60
3       46.90
4       46.90
        ...  
2864    49.20
2865    51.08
2866    51.73
2867    51.96
2868    54.95
Name: 전용면적, Length: 2869, dtype: float64

In [39]:
# 10의 자리 이하 버림
train_m["전용면적"] = train_m["전용면적"]// 10*10
print(train_m["전용면적"])
# 100보다 크면 100으로
train_m["전용면적"] = np.where(train_m["전용면적"] > 100, 100, train_m["전용면적"])
# 15보다 작으면 15로
train_m["전용면적"] = np.where(train_m['전용면적'] <15, 15, train_m['전용면적'])

0       30.0
1       30.0
2       30.0
3       40.0
4       40.0
        ... 
2864    40.0
2865    50.0
2866    50.0
2867    50.0
2868    50.0
Name: 전용면적, Length: 2869, dtype: float64


In [40]:
train_m.loc[:,'총전용면적'] = train_m.loc[:,'전용면적'] * train_m.loc[:,'전용면적별세대수']

codes = train_m.단지코드.unique()
areas = np.sort(train_m.전용면적.unique())

train = pd.DataFrame()
columns = ['단지코드', '등록차량수', '총세대수', '지역', '공가수','지하철역수', '버스정류장수', '단지내주차면수']

for order, code in enumerate(codes):
    temp_by_code = train_m.loc[train_m.단지코드==code].reset_index(drop=True)
    ## 원 계열 값 그냥 가져오기 
    train.loc[order, columns] = temp_by_code.loc[0, columns]             
    train.loc[order, "총임대가구수"] = temp_by_code.전용면적별세대수.sum()

    for area in areas:
        temp_by_code_areas = temp_by_code.loc[temp_by_code.전용면적==area].reset_index(drop=True)

        if temp_by_code_areas.shape[0] !=0:
            train.loc[order, f'면적_{int(area)}'] = temp_by_code_areas.전용면적별세대수.sum() / temp_by_code_areas.총세대수[0]
        else:
            train.loc[order, f'면적_{int(area)}'] = 0

train["임대비율"] = train.총임대가구수 / train.총세대수
train["가구당주차면수"] = train.단지내주차면수 / train.총세대수


In [41]:
train

,단지코드,등록차량수,총세대수,지역,공가수,지하철역수,버스정류장수,단지내주차면수,총임대가구수,면적_15,면적_20,면적_30,면적_40,면적_50,면적_60,면적_70,면적_80,면적_100,임대비율,가구당주차면수
0,C2515,205.0,545.0,경상남도,17.0,0.0,3.0,624.0,545.0,0.000000,0.000000,0.653211,0.104587,0.242202,0.0,0.0,0.0,0.0,1.0,1.144954
1,C1407,1064.0,1216.0,대전광역시,13.0,1.0,1.0,1285.0,1216.0,0.000000,0.000000,0.320724,0.577303,0.101974,0.0,0.0,0.0,0.0,1.0,1.056743
2,C1945,730.0,755.0,경기도,6.0,1.0,3.0,734.0,755.0,0.000000,0.000000,0.317881,0.000000,0.682119,0.0,0.0,0.0,0.0,1.0,0.972185
3,C1470,553.0,696.0,전라북도,14.0,0.0,2.0,645.0,696.0,0.000000,0.000000,0.364943,0.281609,0.353448,0.0,0.0,0.0,0.0,1.0,0.926724
4,C1898,415.0,566.0,전라북도,9.0,0.0,6.0,517.0,566.0,0.000000,0.000000,0.478799,0.369258,0.151943,0.0,0.0,0.0,0.0,1.0,0.913428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,C2586,57.0,90.0,제주특별자치도,7.0,0.0,3.0,66.0,90.0,0.133333,0.466667,0.400000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.733333
407,C2035,246.0,492.0,강원도,24.0,0.0,1.0,521.0,492.0,0.000000,0.317073,0.317073,0.365854,0.000000,0.0,0.0,0.0,0.0,1.0,1.058943
408,C2020,19.0,40.0,부산광역시,7.0,1.0,2.0,25.0,40.0,0.500000,0.125000,0.375000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.625000
409,C2437,16.0,90.0,충청북도,12.0,0.0,1.0,30.0,90.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.333333


#### test

In [42]:
test_m["전용면적"]

0       39.79
1       46.81
2       46.90
3       46.90
4       51.46
        ...  
1003    36.77
1004    29.19
1005    29.19
1006    39.45
1007    46.23
Name: 전용면적, Length: 1008, dtype: float64

In [43]:
# 10의 자리 이하 버림
test_m["전용면적"] = test_m["전용면적"]// 10*10
# 100보다 크면 100으로
test_m["전용면적"] = np.where(test_m["전용면적"] > 100, 100, test_m["전용면적"])
# 15보다 작으면 15로
test_m["전용면적"] = np.where(test_m['전용면적'] <15, 15, test_m['전용면적'])

In [44]:
test_m.loc[:,'총전용면적'] = test_m.loc[:,'전용면적'] * test_m.loc[:,'전용면적별세대수']

codes = test_m.단지코드.unique()
areas = np.sort(test_m.전용면적.unique())

test = pd.DataFrame()
columns = ['단지코드', '총세대수', '지역', '공가수','지하철역수', '버스정류장수', '단지내주차면수']

for order, code in enumerate(codes):
    temp_by_code = test_m.loc[test_m.단지코드==code].reset_index(drop=True)
    ## 원 계열 값 그냥 가져오기 
    test.loc[order, columns] = temp_by_code.loc[0, columns]             
    test.loc[order, "총임대가구수"] = temp_by_code.전용면적별세대수.sum()

    for area in areas:
        temp_by_code_areas = temp_by_code.loc[temp_by_code.전용면적==area].reset_index(drop=True)

        if temp_by_code_areas.shape[0] !=0:
            test.loc[order, f'면적_{int(area)}'] = temp_by_code_areas.전용면적별세대수.sum() / temp_by_code_areas.총세대수[0]
        else:
            test.loc[order, f'면적_{int(area)}'] = 0

test["임대비율"] = test.총임대가구수 / test.총세대수
test["가구당주차면수"] = test.단지내주차면수 / test.총세대수


In [45]:
train

,단지코드,등록차량수,총세대수,지역,공가수,지하철역수,버스정류장수,단지내주차면수,총임대가구수,면적_15,면적_20,면적_30,면적_40,면적_50,면적_60,면적_70,면적_80,면적_100,임대비율,가구당주차면수
0,C2515,205.0,545.0,경상남도,17.0,0.0,3.0,624.0,545.0,0.000000,0.000000,0.653211,0.104587,0.242202,0.0,0.0,0.0,0.0,1.0,1.144954
1,C1407,1064.0,1216.0,대전광역시,13.0,1.0,1.0,1285.0,1216.0,0.000000,0.000000,0.320724,0.577303,0.101974,0.0,0.0,0.0,0.0,1.0,1.056743
2,C1945,730.0,755.0,경기도,6.0,1.0,3.0,734.0,755.0,0.000000,0.000000,0.317881,0.000000,0.682119,0.0,0.0,0.0,0.0,1.0,0.972185
3,C1470,553.0,696.0,전라북도,14.0,0.0,2.0,645.0,696.0,0.000000,0.000000,0.364943,0.281609,0.353448,0.0,0.0,0.0,0.0,1.0,0.926724
4,C1898,415.0,566.0,전라북도,9.0,0.0,6.0,517.0,566.0,0.000000,0.000000,0.478799,0.369258,0.151943,0.0,0.0,0.0,0.0,1.0,0.913428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,C2586,57.0,90.0,제주특별자치도,7.0,0.0,3.0,66.0,90.0,0.133333,0.466667,0.400000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.733333
407,C2035,246.0,492.0,강원도,24.0,0.0,1.0,521.0,492.0,0.000000,0.317073,0.317073,0.365854,0.000000,0.0,0.0,0.0,0.0,1.0,1.058943
408,C2020,19.0,40.0,부산광역시,7.0,1.0,2.0,25.0,40.0,0.500000,0.125000,0.375000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.625000
409,C2437,16.0,90.0,충청북도,12.0,0.0,1.0,30.0,90.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.333333


In [46]:
test

,단지코드,총세대수,지역,공가수,지하철역수,버스정류장수,단지내주차면수,총임대가구수,면적_15,면적_20,면적_30,면적_40,면적_50,면적_60,면적_70,면적_80,면적_100,임대비율,가구당주차면수
0,C1072,754.0,경기도,14.0,0.0,2.0,683.0,754.0,0.000000,0.000000,0.153846,0.347480,0.498674,0.0,0.0,0.0,0.0,1.000000,0.905836
1,C1128,1354.0,경기도,9.0,0.0,3.0,1216.0,1354.0,0.000000,0.000000,0.310192,0.262925,0.426883,0.0,0.0,0.0,0.0,1.000000,0.898080
2,C1456,619.0,부산광역시,18.0,0.0,16.0,547.0,619.0,0.000000,0.000000,0.399031,0.316640,0.284330,0.0,0.0,0.0,0.0,1.000000,0.883683
3,C1840,593.0,전라북도,7.0,0.0,3.0,543.0,593.0,0.000000,0.000000,0.564924,0.293423,0.141653,0.0,0.0,0.0,0.0,1.000000,0.915683
4,C1332,1297.0,경기도,11.0,0.0,2.0,1112.0,1297.0,0.000000,0.000000,0.327679,0.437934,0.234387,0.0,0.0,0.0,0.0,1.000000,0.857363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,C2456,349.0,제주특별자치도,17.0,0.0,4.0,270.0,346.0,0.000000,0.320917,0.395415,0.275072,0.000000,0.0,0.0,0.0,0.0,0.991404,0.773639
143,C1266,596.0,충청북도,35.0,0.0,1.0,593.0,591.0,0.000000,0.302013,0.510067,0.179530,0.000000,0.0,0.0,0.0,0.0,0.991611,0.994966
144,C2152,120.0,강원도,9.0,0.0,1.0,40.0,120.0,0.000000,0.550000,0.450000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.333333
145,C1267,675.0,경상남도,38.0,0.0,1.0,467.0,670.0,0.074074,0.465185,0.186667,0.266667,0.000000,0.0,0.0,0.0,0.0,0.992593,0.691852


***총임대가구수와 공가수.... ***

### 3.2.2 임대건물구분
- 상가를 포함한 단지와 포함하지 않은 단지 구분? ***상가가 있는곳은 주차공간이 넓어야하지않나? 싶어서***
- 상가만 존재하는 단지가 있나? => x
- 따라서 상가 포함 단지와 상가 아파트 단독 단지를 구분
- 아파트 단독(1), 상가포함(2)

In [47]:
# 상가 보유 단지
print('전체 단지 수: ', train_m['단지코드'].nunique())
print('상가 보유 단지 수: ', train_m.loc[train_m['임대건물구분'] == '상가']['단지코드'].nunique())
print('상가 보유 단지: ', train_m.loc[train_m['임대건물구분'] == '상가']['단지코드'].unique())
print('상가 보유 단지 지역: ', train_m.loc[train_m['임대건물구분'] == '상가']['지역'].unique())

전체 단지 수:  411
상가 보유 단지 수:  33
상가 보유 단지:  ['C1925' 'C1874' 'C2416' 'C2621' 'C1616' 'C1704' 'C2258' 'C2038' 'C1859'
 'C1722' 'C2190' 'C1476' 'C1983' 'C2135' 'C2034' 'C1109' 'C2289' 'C2597'
 'C2310' 'C2132' 'C1439' 'C1899' 'C1056' 'C2644' 'C1206' 'C1775' 'C1790'
 'C2109' 'C1698' 'C1004' 'C1875' 'C2212' 'C2571']
상가 보유 단지 지역:  ['강원도' '충청남도' '경상남도' '대전광역시' '부산광역시' '제주특별자치도']


In [48]:
# 상가 보유 단지의 리스트로 상가만 존재하는 단지가 있는지 확인
temp_tr = ['C1925', 'C1874', 'C2416', 'C2621', 'C1616', 'C1704', 'C2258', 'C2038', 'C1859',
 'C1722', 'C2190', 'C1476', 'C1983', 'C2135', 'C2034', 'C1109', 'C2289', 'C2597',
 'C2310', 'C2132', 'C1439', 'C1899', 'C1056', 'C2644', 'C1206', 'C1775', 'C1790',
 'C2109', 'C1698', 'C1004', 'C1875', 'C2212', 'C2571']

for i in temp_tr:
    print(i)
    display(train_m.loc[train_m["단지코드"] == i]["임대건물구분"].unique())

C1925


array(['아파트', '상가'], dtype=object)

C1874


array(['아파트', '상가'], dtype=object)

C2416


...

array(['아파트', '상가'], dtype=object)

C2212


array(['아파트', '상가'], dtype=object)

C2571


array(['아파트', '상가'], dtype=object)

In [49]:
train["임대건물구분"] = "아파트"

#상가 보유하고 있으면 1, 아파트만 있으면 0
for i in temp_tr:
    train.loc[train["단지코드"] == i]["임대건물구분"] = "주상복합"

In [50]:
train

,단지코드,등록차량수,총세대수,지역,공가수,지하철역수,버스정류장수,단지내주차면수,총임대가구수,면적_15,...,면적_30,면적_40,면적_50,면적_60,면적_70,면적_80,면적_100,임대비율,가구당주차면수,임대건물구분
0,C2515,205.0,545.0,경상남도,17.0,0.0,3.0,624.0,545.0,0.000000,...,0.653211,0.104587,0.242202,0.0,0.0,0.0,0.0,1.0,1.144954,아파트
1,C1407,1064.0,1216.0,대전광역시,13.0,1.0,1.0,1285.0,1216.0,0.000000,...,0.320724,0.577303,0.101974,0.0,0.0,0.0,0.0,1.0,1.056743,아파트
2,C1945,730.0,755.0,경기도,6.0,1.0,3.0,734.0,755.0,0.000000,...,0.317881,0.000000,0.682119,0.0,0.0,0.0,0.0,1.0,0.972185,아파트
3,C1470,553.0,696.0,전라북도,14.0,0.0,2.0,645.0,696.0,0.000000,...,0.364943,0.281609,0.353448,0.0,0.0,0.0,0.0,1.0,0.926724,아파트
4,C1898,415.0,566.0,전라북도,9.0,0.0,6.0,517.0,566.0,0.000000,...,0.478799,0.369258,0.151943,0.0,0.0,0.0,0.0,1.0,0.913428,아파트
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,C2586,57.0,90.0,제주특별자치도,7.0,0.0,3.0,66.0,90.0,0.133333,...,0.400000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.733333,아파트
407,C2035,246.0,492.0,강원도,24.0,0.0,1.0,521.0,492.0,0.000000,...,0.317073,0.365854,0.000000,0.0,0.0,0.0,0.0,1.0,1.058943,아파트
408,C2020,19.0,40.0,부산광역시,7.0,1.0,2.0,25.0,40.0,0.500000,...,0.375000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.625000,아파트
409,C2437,16.0,90.0,충청북도,12.0,0.0,1.0,30.0,90.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.333333,아파트


In [51]:
# 상가 보유 단지
print('전체 단지 수: ', test_m['단지코드'].nunique())
print('상가 보유 단지 수: ', test_m.loc[test_m['임대건물구분'] == '상가']['단지코드'].nunique())
print('상가 보유 단지: ', test_m.loc[test_m['임대건물구분'] == '상가']['단지코드'].unique())
print('상가 보유 단지 지역: ', test_m.loc[test_m['임대건물구분'] == '상가']['지역'].unique())

전체 단지 수:  147
상가 보유 단지 수:  7
상가 보유 단지:  ['C1006' 'C2676' 'C2177' 'C2033' 'C1812' 'C2253' 'C1729']
상가 보유 단지 지역:  ['대전광역시' '부산광역시' '충청남도' '울산광역시' '강원도']


In [52]:
# 상가 보유 단지의 리스트로 상가만 존재하는 단지가 있는지 확인
temp_te = ['C1006', 'C2676', 'C2177', 'C2033', 'C1812', 'C2253', 'C1729']

for i in temp_te:
    print(i)
    display(test_m.loc[test_m["단지코드"] == i]["임대건물구분"].unique())

C1006


array(['아파트', '상가'], dtype=object)

C2676


array(['아파트', '상가'], dtype=object)

C2177


...

array(['아파트', '상가'], dtype=object)

C2253


array(['아파트', '상가'], dtype=object)

C1729


array(['아파트', '상가'], dtype=object)

In [53]:
test["임대건물구분"] = "아파트"

#상가 보유하고 있으면 1, 아파트만 있으면 0
for i in temp_te:
    test.loc[test["단지코드"] == i]["임대건물구분"] = "주상복합"

In [54]:
test

,단지코드,총세대수,지역,공가수,지하철역수,버스정류장수,단지내주차면수,총임대가구수,면적_15,면적_20,면적_30,면적_40,면적_50,면적_60,면적_70,면적_80,면적_100,임대비율,가구당주차면수,임대건물구분
0,C1072,754.0,경기도,14.0,0.0,2.0,683.0,754.0,0.000000,0.000000,0.153846,0.347480,0.498674,0.0,0.0,0.0,0.0,1.000000,0.905836,아파트
1,C1128,1354.0,경기도,9.0,0.0,3.0,1216.0,1354.0,0.000000,0.000000,0.310192,0.262925,0.426883,0.0,0.0,0.0,0.0,1.000000,0.898080,아파트
2,C1456,619.0,부산광역시,18.0,0.0,16.0,547.0,619.0,0.000000,0.000000,0.399031,0.316640,0.284330,0.0,0.0,0.0,0.0,1.000000,0.883683,아파트
3,C1840,593.0,전라북도,7.0,0.0,3.0,543.0,593.0,0.000000,0.000000,0.564924,0.293423,0.141653,0.0,0.0,0.0,0.0,1.000000,0.915683,아파트
4,C1332,1297.0,경기도,11.0,0.0,2.0,1112.0,1297.0,0.000000,0.000000,0.327679,0.437934,0.234387,0.0,0.0,0.0,0.0,1.000000,0.857363,아파트
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,C2456,349.0,제주특별자치도,17.0,0.0,4.0,270.0,346.0,0.000000,0.320917,0.395415,0.275072,0.000000,0.0,0.0,0.0,0.0,0.991404,0.773639,아파트
143,C1266,596.0,충청북도,35.0,0.0,1.0,593.0,591.0,0.000000,0.302013,0.510067,0.179530,0.000000,0.0,0.0,0.0,0.0,0.991611,0.994966,아파트
144,C2152,120.0,강원도,9.0,0.0,1.0,40.0,120.0,0.000000,0.550000,0.450000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.333333,아파트
145,C1267,675.0,경상남도,38.0,0.0,1.0,467.0,670.0,0.074074,0.465185,0.186667,0.266667,0.000000,0.0,0.0,0.0,0.0,0.992593,0.691852,아파트


In [55]:
train.loc[:,"임대건물구분"] = train.loc[:,"임대건물구분"].astype('category')
test.loc[:,"임대건물구분"] = test.loc[:,"임대건물구분"].astype('category')

### 3.2.3 주거율
- 주거율 : (1 - (공가수 / 총세대수))
- 주거율이 높으면 단지내 주차면수 대비 등록차량의 수의 비율이 높을것이라 예상

In [56]:
# 그 전에 미리 상가의 공가수?가 어떻게 되는지 확인
train_m[train_m["임대건물구분"] == "상가"]["공가수"]

80     9
81     9
82     9
83     9
93     2
      ..
826    5
827    5
828    5
829    5
830    5
Name: 공가수, Length: 562, dtype: int64

In [57]:
train["공가수"] = 1 - (train["공가수"]/train["총세대수"])
test["공가수"] = 1 - (test["공가수"]/test["총세대수"])

In [58]:
train.rename(columns={"공가수" : "주거율"}, inplace=True)
test.rename(columns={"공가수" : "주거율"}, inplace=True)

In [59]:
display(train.head(10),test.head(10))

,단지코드,등록차량수,총세대수,지역,주거율,지하철역수,버스정류장수,단지내주차면수,총임대가구수,면적_15,...,면적_30,면적_40,면적_50,면적_60,면적_70,면적_80,면적_100,임대비율,가구당주차면수,임대건물구분
0,C2515,205.0,545.0,경상남도,0.968807,0.0,3.0,624.0,545.0,0.0,...,0.653211,0.104587,0.242202,0.0,0.0,0.0,0.0,1.0,1.144954,아파트
1,C1407,1064.0,1216.0,대전광역시,0.989309,1.0,1.0,1285.0,1216.0,0.0,...,0.320724,0.577303,0.101974,0.0,0.0,0.0,0.0,1.0,1.056743,아파트
2,C1945,730.0,755.0,경기도,0.992053,1.0,3.0,734.0,755.0,0.0,...,0.317881,0.000000,0.682119,0.0,0.0,0.0,0.0,1.0,0.972185,아파트
3,C1470,553.0,696.0,전라북도,0.979885,0.0,2.0,645.0,696.0,0.0,...,0.364943,0.281609,0.353448,0.0,0.0,0.0,0.0,1.0,0.926724,아파트
4,C1898,415.0,566.0,전라북도,0.984099,0.0,6.0,517.0,566.0,0.0,...,0.478799,0.369258,0.151943,0.0,0.0,0.0,0.0,1.0,0.913428,아파트
5,C1244,1804.0,1722.0,경기도,0.994193,0.0,10.0,1483.0,1722.0,0.0,...,0.457607,0.315912,0.226481,0.0,0.0,0.0,0.0,1.0,0.861208,아파트
6,C1171,700.0,624.0,대전광역시,0.979167,0.0,5.0,634.0,624.0,0.0,...,0.567308,0.368590,0.064103,0.0,0.0,0.0,0.0,1.0,1.016026,아파트
7,C2073,301.0,361.0,강원도,0.963989,0.0,5.0,288.0,361.0,0.0,...,0.465374,0.423823,0.110803,0.0,0.0,0.0,0.0,1.0,0.797784,아파트
8,C2513,820.0,754.0,광주광역시,0.980106,0.0,6.0,530.0,754.0,0.0,...,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,1.0,0.702918,아파트
9,C1936,234.0,240.0,광주광역시,0.966667,0.0,10.0,168.0,240.0,0.0,...,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,1.0,0.700000,아파트


,단지코드,총세대수,지역,주거율,지하철역수,버스정류장수,단지내주차면수,총임대가구수,면적_15,면적_20,면적_30,면적_40,면적_50,면적_60,면적_70,면적_80,면적_100,임대비율,가구당주차면수,임대건물구분
0,C1072,754.0,경기도,0.981432,0.0,2.0,683.0,754.0,0.0,0.000000,0.153846,0.347480,0.498674,0.0,0.0,0.0,0.0,1.0,0.905836,아파트
1,C1128,1354.0,경기도,0.993353,0.0,3.0,1216.0,1354.0,0.0,0.000000,0.310192,0.262925,0.426883,0.0,0.0,0.0,0.0,1.0,0.898080,아파트
2,C1456,619.0,부산광역시,0.970921,0.0,16.0,547.0,619.0,0.0,0.000000,0.399031,0.316640,0.284330,0.0,0.0,0.0,0.0,1.0,0.883683,아파트
3,C1840,593.0,전라북도,0.988196,0.0,3.0,543.0,593.0,0.0,0.000000,0.564924,0.293423,0.141653,0.0,0.0,0.0,0.0,1.0,0.915683,아파트
4,C1332,1297.0,경기도,0.991519,0.0,2.0,1112.0,1297.0,0.0,0.000000,0.327679,0.437934,0.234387,0.0,0.0,0.0,0.0,1.0,0.857363,아파트
5,C1563,1974.0,경기도,0.992401,0.0,6.0,1696.0,1974.0,0.0,0.000000,0.364235,0.332320,0.303445,0.0,0.0,0.0,0.0,1.0,0.859169,아파트
6,C1794,1349.0,전라북도,0.981468,0.0,2.0,1098.0,1349.0,0.0,0.181616,0.486286,0.222387,0.109711,0.0,0.0,0.0,0.0,1.0,0.813936,아파트
7,C1640,533.0,전라북도,0.968105,0.0,1.0,470.0,533.0,0.0,0.000000,0.163227,0.000000,0.836773,0.0,0.0,0.0,0.0,1.0,0.881801,아파트
8,C1377,470.0,경상남도,0.961702,0.0,4.0,384.0,470.0,0.0,0.131915,0.497872,0.148936,0.221277,0.0,0.0,0.0,0.0,1.0,0.817021,아파트
9,C2072,353.0,경상남도,0.983003,0.0,1.0,280.0,353.0,0.0,0.000000,0.583569,0.161473,0.254958,0.0,0.0,0.0,0.0,1.0,0.793201,아파트


*** 공가수? 임대비율? ***

### 3.2.4 공급유형

In [60]:
train_o[train_o["공급유형"] != "임대상가"]
#장기전세, 공공분양, 국민임대, 행복주택

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철역수,버스정류장수,단지내주차면수
2073,C1039,790,아파트,서울특별시,장기전세,51.32,126,13,A,187694000,0,0,3,673
2074,C1039,790,아파트,서울특별시,장기전세,59.88,49,13,A,213863000,0,0,3,673
2075,C1039,790,아파트,서울특별시,장기전세,59.94,75,13,A,213863000,0,0,3,673
2284,C1350,1401,아파트,대전광역시,공공분양,74.94,317,2,D,0,0,0,6,1636
2285,C1350,1401,아파트,대전광역시,공공분양,74.94,137,2,D,0,0,0,6,1636
2286,C1350,1401,아파트,대전광역시,공공분양,74.94,22,2,D,0,0,0,6,1636
2287,C1350,1401,아파트,대전광역시,공공분양,84.94,164,2,D,0,0,0,6,1636
2288,C1350,1401,아파트,대전광역시,공공분양,84.94,19,2,D,0,0,0,6,1636
2289,C1350,1401,아파트,대전광역시,공공분양,84.96,26,2,D,0,0,0,6,1636
2290,C1350,1401,아파트,대전광역시,공공분양,84.97,26,2,D,0,0,0,6,1636


In [63]:
train_m.loc[train_m["단지코드"] == "C2186",:"단지내주차면수"]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철역수,버스정류장수,단지내주차면수
2823,C2186,924,아파트,대구광역시,국민임대,20.0,238,0,H,0,0,0,8,664
2824,C2186,924,아파트,대구광역시,국민임대,20.0,38,0,H,0,0,0,8,664
2825,C2186,924,아파트,대구광역시,국민임대,30.0,220,0,H,0,0,0,8,664
2826,C2186,924,아파트,대구광역시,국민임대,40.0,188,0,H,25981000,248630,0,8,664
2827,C2186,924,아파트,대구광역시,영구임대,20.0,219,0,C,13126000,105870,0,8,664
2828,C2186,924,아파트,대구광역시,영구임대,20.0,13,0,C,13164000,106190,0,8,664


### 3.2.5 자격유형